In [ ]:
using  SymPy
using  Plots;pyplot()
using LinearAlgebra

### Input the elastic properties of matrix and inclusion

In [ ]:
Eᵢ = 37.7e3
Eₘ = 377.7e3
νᵢ = 0.22
νₘ = 0.2
@show λᵢ = Eᵢ*νᵢ/((1+νᵢ)*(1-2*νᵢ))
@show λₘ = Eₘ*νₘ/((1+νₘ)*(1-2*νₘ))
@show μᵢ = Eᵢ/(2*(1+νᵢ))
@show μₘ = Eₘ/(2*(1+νₘ))

### Dimensions of the Plate

In [ ]:
L = 20
H = 20
h = 0.5  # Diffsed length
ht = 1

### Defining the symbolic variable using SymPy

In [ ]:
x,x₁,y,y₁ = symbols("x,x₁,y,y₁", real = true)

### Piecewise function

In [ ]:
heaviside(x) = 0.5 * (sign(x) + 1)

interval(x, a, b) = heaviside(x-a) - heaviside(x-b)

### Gaussian Kernel Function

In [ ]:
wₕ(x,x₁) = (1/(sqrt(2*pi)*h))*exp(-(x-x₁)^2/(2*h^2))

### Discontinuous material properties

In [ ]:
λFx1 = interval(x₁,-L-ht,L+ht)*wₕ(x,x₁)
λFy1 = interval(y₁,-H-ht,0)*wₕ(y,y₁)
λFy2 = interval(y₁,0,H+ht)*wₕ(y,y₁)

In [ ]:
μFx1 = interval(x₁,-L-ht,L+ht)*wₕ(x,x₁)
μFy1 = interval(y₁,-H-ht,0)*wₕ(y,y₁)
μFy2 = interval(y₁,0,H+ht)*wₕ(y,y₁)

In [ ]:
λdiscont1 = λᵢ * interval(x₁,-L-ht,L+ht)* interval(y₁,-H-ht,0)
λdiscont2 = λₘ * interval(x₁,-L-ht,L+ht)* interval(y₁,0,H+ht)
λdiscont = λdiscont1 + λdiscont2

In [ ]:
μdiscont1 = μᵢ * interval(x₁,-L-ht,L+ht)* interval(y₁,-H-ht,0)
μdiscont2 = μₘ * interval(x₁,-L-ht,L+ht)* interval(y₁,0,H+ht)
μdiscont = μdiscont1 + μdiscont2

### Diffused material properties

In [ ]:
smoothλ1 = integrate(λFx1,(x₁,-L-ht,L+ht))*integrate(λFy1,(y₁,-H-ht,H+ht))*λᵢ
smoothλ2 = integrate(λFx1,(x₁,-L-ht,L+ht))*integrate(λFy2,(y₁,-H-ht,H+ht))*λₘ
smoothλ = smoothλ1 + smoothλ2

In [ ]:
smoothμ1 = integrate(μFx1,(x₁,-L-ht,L+ht))*integrate(μFy1,(y₁,-H-ht,H+ht))*μᵢ
smoothμ2 = integrate(μFx1,(x₁,-L-ht,L+ht))*integrate(μFy2,(y₁,-H-ht,H+ht))*μₘ
smoothμ = smoothμ1 + smoothμ2

## Illustration

In [ ]:
xs = range(-L,L, length=100)
ys = range(-H,H, length=100)

plot1 = contourf(xs, ys, λdiscont, c=:diverging_bwr_40_95_c42_n256, axis = nothing, showaxis =false,aspect_ratio=:equal, camera=(0,90))
plot2 = contourf(xs, ys, smoothλ, c=:diverging_bwr_40_95_c42_n256, axis = nothing, showaxis =false,aspect_ratio=:equal,camera=(0,90))
plot3 = contourf(xs, ys, μdiscont, c=:diverging_bwr_40_95_c42_n256, axis = nothing, showaxis =false,aspect_ratio=:equal, camera=(0,90))
plot4 = contourf(xs, ys, smoothμ, c=:diverging_bwr_40_95_c42_n256, axis = nothing, showaxis =false,aspect_ratio=:equal, camera=(0,90))
plotλ = plot(plot1,plot2)
plotμ = plot(plot3,plot4)

### Save figures

In [ ]:
savefig(plotλ, "TwoMaterialλDiffusion.pdf")
savefig(plotμ, "TwoMaterialμDiffusion.pdf")